# Notebook para compilar **GROMACS** en el Colab

Este notebook descarga la versión elegida de [GROMACS desde la web oficial](ftp://ftp.gromacs.org/gromacs), y la compila usando los flags del usuario



# 1. Carga del drive

Primero se carga el drive y se crea el directorio con el nombre de la variable ```gromacs_dir```, la cual por defecto es ```gromacs_md```, aunque puede ser cambiada por el usuario.

In [ ]:
import os

if not os.path.isdir("/content/drive/MyDrive"):
  from google.colab import drive
  drive.mount("/content/drive")

if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")

gromacs_dir = "gromacs_md" #@param {type: "string"}
storage = f"/content/drive/MyDrive/{gromacs_dir}"
os.makedirs(storage, exist_ok=True)
%env STORAGE={storage}

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}

# 2. Compilación de GROMACS

A continuación se compila **GROMACS** con la versión seleccionada en ```gromacs_vers```, por defecto compila la versión **2024.1**, los flags se pasan en ```gromacs_flags``` los cuales por defecto son ```-DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA``` pero pueden pasarse otros a elección del usuario, para más información sobre la compilación de **GROMACS** ver el [manual oficial](https://manual.gromacs.org/current/index.html).

Por último, la versión compilada se almacena en el directorio creado en el drive en el paso anterior, este puede usarse en otros notebooks de colab.

In [ ]:
%%bash

gromacs_vers="2024.1" #@param {type: "string"}
gromacs_flags="-DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"

wget -q "ftp://ftp.gromacs.org/gromacs/gromacs-${gromacs_vers}.tar.gz"
if [[ ! -s "gromacs-${gromacs_vers}.tar.gz" ]]; then
  echo "Error: could not download: gromacs-${gromacs_vers}.tar.gz" >&2
  exit 1
fi
tar -xzf "gromacs-${gromacs_vers}.tar.gz"
rm "gromacs-${gromacs_vers}.tar.gz"
cd "gromacs-${gromacs_vers}"
mkdir "build"
cd "build"
cmake .. "${gromacs_flags}"
make -j $(nproc)
make install # -> /usr/local/gromacs
# Cache
tar -czf "my_gromacs.tar.gz" -C "/usr/local" "gromacs"
mv "my_gromacs.tar.gz" "${cache_gromacs}"

# 3. Pueba de compilación

Esta celda prueba que la compilación sea correcta

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx -h